# Using the OpenAI API

First, install the openai module if it is not available.

In [ ]:
%pip install openai
%pip install python-dotenv

## Setting up the environment

In this notebook, we will be getting the OpenAI API key from the environment. Make sure the directory contains a `.env` file with your key. 

In [ ]:
import os
import openai
from openai import OpenAI
import dotenv
dotenv.load_dotenv(".env", override=True) 

openai.api_key = os.getenv("OPENAI_API_KEY")

## Creating a first prompt

In [ ]:
MODELS = ["gpt-3.5-turbo-1106", "gpt-4", "gpt-4o", "gpt-3.5-turbo-16k"]
MODEL = MODELS[0]
client = OpenAI()

def query_openai(client, prompt, model):
    response = client.chat.completions.create(
        model=model,
            messages=[{
                "role": "system",
                "content": "You will be asked to help with programming questions."
            },
            {
                "role": "user",
                "content": prompt
            }],
        max_tokens=256
        )
    return(response.choices[0].message.content)


In [ ]:
# Example usage
prompt_text = "Provide some Perl code to write `Hello world` in five different ways."
response_text = query_openai(client, prompt_text, MODEL)
print(response_text)

## Getting data from an API

In this example we will get data from a public API and ask ChatGPT to reason about this.
First, we need the code to access the API. In our case we are looking at Gene Ontology database, but you can use another API as well of course.
You can do this through the ChatGPT web interface, but for now we will stick to the API.

In [ ]:
prompt_text = "I need to query the gene ontology database through the API. Can you provide code which gives information on GO:0030445?"
response_text = query_openai(client, prompt_text, MODEL)
print(response_text)

In [ ]:
## Accessing the API

In [ ]:
%pip install requests

Let's see if it works

In [ ]:
import requests

# Make a GET request to the gene ontology API
response = requests.get('http://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/GO:0030445')

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()
    
    # Print the information for GO:0030445
    print("GO Term ID:", data['results'][0]['id'])
    print("GO Term Name:", data['results'][0]['name'])
    print("GO Term Definition:", data['results'][0]['definition']['text'])
else:
    print("Failed to get information for GO:0030445")

Now create a function to get all information.

In [ ]:
def get_go(GOterm):
    # Make a GET request to the gene ontology API
    response = requests.get("http://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{}".format(GOterm))

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        return data
    else:
        print("Failed to get information for {}".format(GOterm))

Let's have ChatGPT make a summary of the data. First we need a more general function to query the OpenAI API.

In [ ]:
def query_openai(client, system, prompt, model):
    response = client.chat.completions.create(
        model=model,
            messages=[{
                "role": "system",
                "content": system
            },
            {
                "role": "user",
                "content": prompt
            }],
        max_tokens=256
        )
    return(response.choices[0].message.content)


Let's put them together.

In [ ]:
my_go_term = "GO:0030445"
system_prompt = "You are a skilled biologist and a good lecturer."

go_json = get_go(my_go_term)
summary = query_openai(client=client, 
                       system=system_prompt, 
                       prompt="I have this json from the Gene Ontology database. Could you create a nice summary of the biological data? There is no need to comment on the structure of the JSON: `{}`".format(go_json), 
                       model=MODEL)
print(summary)

In [ ]:
report = query_openai(client=client,
                      system=system_prompt,
                      prompt="Given the summary of this GO term, could you provide a markdown report on this GO term with more background information? Please only markdown, no other comments or explainations. `{}`".format(summary),
                      model=MODEL)
print(report)